# Start session

In [1]:
import activeviam as av
import pandas as pd

In [2]:
session = av.create_session("Sales Booster")

# Load data in the cube

In [3]:
sales = session.read_csv("data/projected_sales.csv", keys=["Customer", "Salesperson"], store_name="Projected sales")
sales.head()

,Customer,Salesperson,Continent,Country,Amount
0,Ubisoft,Léodagan,Europe,France,100000.0
1,Decathlon,Lancelot,Europe,France,150000.0
2,Airbus,Karadoc,Europe,France,80000.0
3,EDF,Bohort,Europe,France,120000.0
4,Air France,Lancelot,Europe,France,70000.0


In [4]:
salespeople_df = pd.read_csv("data/salespeople.csv")
salespeople_df

,Name,Position,Quota,Rate,Boosted Rate
0,Lancelot,Executive,550000.0,0.025,0.030
1,Perceval,Associate,450000.0,0.015,0.025
2,Karadoc,Associate,450000.0,0.015,0.025
3,Léodagan,Senior Associate,450000.0,0.020,0.020
4,Bohort,Junior Representative,300000.0,0.010,0.020


In [5]:
salespeople = session.read_pandas(salespeople_df, keys=["Name"], store_name="Salespeople")
salespeople

Store(name='Salespeople', columns='Name, Position, Quota, Rate, Boosted Rate')

In [6]:
sales.join(salespeople, mapping={"Salesperson" : "Name"})

In [7]:
cube = session.create_cube(sales, name="Sales")
h = cube.hierarchies
lvl = cube.levels
m = cube.measures
cube

Cube(measures=MeasuresManager(measures="NamedMeasure(name = 'Rate.AVG', underlying=None),NamedMeasure(name = 'Boosted Rate.AVG', underlying=None),NamedMeasure(name = 'contributors.COUNT', underlying=None),NamedMeasure(name = 'Quota.SUM', underlying=None),NamedMeasure(name = 'Amount.SUM', underlying=None),NamedMeasure(name = 'Rate.VALUE', underlying=None),NamedMeasure(name = 'Rate.SUM', underlying=None),NamedMeasure(name = 'Boosted Rate.VALUE', underlying=None),NamedMeasure(name = 'Boosted Rate.SUM', underlying=None),NamedMeasure(name = 'Quota.VALUE', underlying=None),NamedMeasure(name = 'Amount.AVG', underlying=None),NamedMeasure(name = 'Quota.AVG', underlying=None)"), hierarchies=HierarchiesManager(hierarchies={'Continent': Hierarchy(name='Continent', levels={'Continent': Level(name='Continent', hierarchy='Continent', comparator=None, property_name='Continent')}, dimension='Hierarchies', slicing=False), 'Position': Hierarchy(name='Position', levels={'Position': Level(name='Position', hierarchy='Position', comparator=None, property_name='Position')}, dimension='Hierarchies', slicing=False), 'Customer': Hierarchy(name='Customer', levels={'Customer': Level(name='Customer', hierarchy='Customer', comparator=None, property_name='Customer')}, dimension='Hierarchies', slicing=False), 'Country': Hierarchy(name='Country', levels={'Country': Level(name='Country', hierarchy='Country', comparator=None, property_name='Country')}, dimension='Hierarchies', slicing=False), 'Salesperson': Hierarchy(name='Salesperson', levels={'Salesperson': Level(name='Salesperson', hierarchy='Salesperson', comparator=None, property_name='Salesperson')}, dimension='Hierarchies', slicing=False), 'Name': Hierarchy(name='Name', levels={'Name': Level(name='Name', hierarchy='Name', comparator=None, property_name='Name')}, dimension='Hierarchies', slicing=False)}))

# Restructure the cube

In [8]:
h["Geography"] = [ lvl["Continent"], lvl["Country"] ]
del h["Continent"]
del h["Country"]

In [9]:
cube

Cube(measures=MeasuresManager(measures="NamedMeasure(name = 'Rate.AVG', underlying=None),NamedMeasure(name = 'Boosted Rate.AVG', underlying=None),NamedMeasure(name = 'contributors.COUNT', underlying=None),NamedMeasure(name = 'Quota.SUM', underlying=None),NamedMeasure(name = 'Amount.SUM', underlying=None),NamedMeasure(name = 'Rate.VALUE', underlying=None),NamedMeasure(name = 'Rate.SUM', underlying=None),NamedMeasure(name = 'Boosted Rate.VALUE', underlying=None),NamedMeasure(name = 'Boosted Rate.SUM', underlying=None),NamedMeasure(name = 'Quota.VALUE', underlying=None),NamedMeasure(name = 'Amount.AVG', underlying=None),NamedMeasure(name = 'Quota.AVG', underlying=None)"), hierarchies=HierarchiesManager(hierarchies={'Geography': Hierarchy(name='Geography', levels={'Continent': Level(name='Continent', hierarchy='Geography', comparator=None, property_name='Continent'), 'Country': Level(name='Country', hierarchy='Geography', comparator=None, property_name='Country')}, dimension='Hierarchies', slicing=False), 'Position': Hierarchy(name='Position', levels={'Position': Level(name='Position', hierarchy='Position', comparator=None, property_name='Position')}, dimension='Hierarchies', slicing=False), 'Customer': Hierarchy(name='Customer', levels={'Customer': Level(name='Customer', hierarchy='Customer', comparator=None, property_name='Customer')}, dimension='Hierarchies', slicing=False), 'Salesperson': Hierarchy(name='Salesperson', levels={'Salesperson': Level(name='Salesperson', hierarchy='Salesperson', comparator=None, property_name='Salesperson')}, dimension='Hierarchies', slicing=False), 'Name': Hierarchy(name='Name', levels={'Name': Level(name='Name', hierarchy='Name', comparator=None, property_name='Name')}, dimension='Hierarchies', slicing=False)}))

# Define Metrics

In [10]:
m["Amount inside quota"] = av.min(m['Amount.SUM'], m['Quota.VALUE'])
m['Regular commission'] = m["Amount inside quota"] * m['Rate.VALUE']
m['Amount above quota'] = av.max(m['Amount.SUM'] - m['Quota.VALUE'], 0)
m['Boosted commission'] = m['Amount above quota'] * m["Boosted Rate.VALUE"]
m['Commission'] = av.agg.sum(m['Regular commission'] + m['Boosted commission'], on=['Salesperson'])

# Adapt quotas and rates

In [11]:
salespeople_df

,Name,Position,Quota,Rate,Boosted Rate
0,Lancelot,Executive,550000.0,0.025,0.030
1,Perceval,Associate,450000.0,0.015,0.025
2,Karadoc,Associate,450000.0,0.015,0.025
3,Léodagan,Senior Associate,450000.0,0.020,0.020
4,Bohort,Junior Representative,300000.0,0.010,0.020


In [12]:
df = salespeople_df.copy(deep=True)
df.loc[df.Name == "Lancelot", "Quota"] = df.Quota + 250000
df.loc[df.Name == "Lancelot", "Rate"] = df.Rate - 0.01
df.loc[df.Name == "Lancelot", "Boosted Rate"] = df["Boosted Rate"] + 0.02
df.loc[df.Position == "Associate", "Rate"] = df.Rate + 0.0085
df

,Name,Position,Quota,Rate,Boosted Rate
0,Lancelot,Executive,800000.0,0.0150,0.050
1,Perceval,Associate,450000.0,0.0235,0.025
2,Karadoc,Associate,450000.0,0.0235,0.025
3,Léodagan,Senior Associate,450000.0,0.0200,0.020
4,Bohort,Junior Representative,300000.0,0.0100,0.020


In [13]:
salespeople.scenarios["Adapted"].load_pandas(df)

# Share

In [14]:
session.url

Url(_Url__url='https://tde-40activeviam-2ecom-1337.notebooks.activeviam.com')